In [ ]:
import os

import pandas as pd

from utils import stringify_input

In [2]:
ROOT_PATH = os.getcwd()

TRAIN_PATH = os.path.join(ROOT_PATH, "map-charting-student-math-misunderstandings", "train.csv")
TEST_PATH = os.path.join(ROOT_PATH, "map-charting-student-math-misunderstandings", "test.csv")

In [3]:
train_df = pd.read_csv(TRAIN_PATH)

In [4]:
train_df.columns

Index(['row_id', 'QuestionId', 'QuestionText', 'MC_Answer',
       'StudentExplanation', 'Category', 'Misconception'],
      dtype='object')

In [5]:
train_df.dtypes

row_id                 int64
QuestionId             int64
QuestionText          object
MC_Answer             object
StudentExplanation    object
Category              object
Misconception         object
dtype: object

In [6]:
train_df.head()

,row_id,QuestionId,QuestionText,MC_Answer,StudentExplanation,Category,Misconception
0,0,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),0ne third is equal to tree nineth,True_Correct,NaN
1,1,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 / 3 because 6 over 9 is 2 thirds and 1 third...,True_Correct,NaN
2,2,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),"1 3rd is half of 3 6th, so it is simplee to un...",True_Neither,NaN
3,3,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 goes into everything and 3 goes into nine,True_Neither,NaN
4,4,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 out of every 3 isn't coloured,True_Correct,NaN


In [7]:
unique_question_ids = train_df.QuestionId.unique()
print(f"Total unique questions: {len(unique_question_ids)}")

Total unique questions: 15


In [8]:
from IPython.display import display, Latex

# GET ANSWER CHOICES
tmp = train_df.groupby(["QuestionId", "MC_Answer"]).size().reset_index(name="count")
tmp["rank"] = (
    tmp.groupby("QuestionId")["count"].rank(method="dense", ascending=False).astype(int)
    - 1
)
tmp = tmp.drop("count", axis=1)
tmp = tmp.sort_values(["QuestionId", "rank"])

# DISPLAY QUESTION AND ANSWER CHOICES
Q = tmp.QuestionId.unique()
for q in Q:
    question = train_df.loc[train_df.QuestionId == q].iloc[0].QuestionText
    choices = tmp.loc[tmp.QuestionId == q].MC_Answer.values
    labels = "ABCD"
    choice_str = " ".join(
        [f"({labels[i]}) {choice}" for i, choice in enumerate(choices)]
    )

    print()
    display(Latex(rf"QuestionId {q}: {question}"))
    display(Latex(rf"MC Answers: {choice_str}"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [9]:
train_df.Misconception = train_df.Misconception.fillna("NA")

In [10]:
train_df.Misconception.unique()

array(['NA', 'Incomplete', 'WNB', 'SwapDividend', 'Mult', 'FlipChange',
       'Irrelevant', 'Wrong_Fraction', 'Additive', 'Not_variable',
       'Adding_terms', 'Inverse_operation', 'Inversion', 'Duplication',
       'Wrong_Operation', 'Whole_numbers_larger', 'Longer_is_bigger',
       'Ignores_zeroes', 'Shorter_is_bigger', 'Wrong_fraction',
       'Adding_across', 'Denominator-only_change',
       'Incorrect_equivalent_fraction_addition', 'Division',
       'Subtraction', 'Unknowable', 'Definition', 'Interior', 'Positive',
       'Tacking', 'Wrong_term', 'Firstterm', 'Base_rate',
       'Multiplying_by_4', 'Certainty', 'Scale'], dtype=object)

In [11]:
unique_categories = train_df.Category.unique()
unique_categories

array(['True_Correct', 'True_Neither', 'True_Misconception',
       'False_Neither', 'False_Misconception', 'False_Correct'],
      dtype=object)

In [12]:
train_df["predict"] = train_df.Category + ":" + train_df.Misconception
print(len(train_df.predict.unique()))

65


In [13]:
for category in unique_categories:
    print(f"Category: {category}")
    rows = train_df[train_df.Category == category]
    for row in rows.itertuples(index=True):
        print(f"Question: {row.QuestionText}")
        print(f"Answer: {row.MC_Answer}")
        print(f"Student Explanation: {row.StudentExplanation}")
        print()

        if row.Index >= 2:
            break
    print(end="\n\n")

Category: True_Correct
Question: What fraction of the shape is not shaded? Give your answer in its simplest form. [Image: A triangle split into 9 equal smaller triangles. 6 of them are shaded.]
Answer: \( \frac{1}{3} \)
Student Explanation: 0ne third is equal to tree nineth

Question: What fraction of the shape is not shaded? Give your answer in its simplest form. [Image: A triangle split into 9 equal smaller triangles. 6 of them are shaded.]
Answer: \( \frac{1}{3} \)
Student Explanation: 1 / 3 because 6 over 9 is 2 thirds and 1 third is not shaded.

Question: What fraction of the shape is not shaded? Give your answer in its simplest form. [Image: A triangle split into 9 equal smaller triangles. 6 of them are shaded.]
Answer: \( \frac{1}{3} \)
Student Explanation: 1 out of every 3 isn't coloured



Category: True_Neither
Question: What fraction of the shape is not shaded? Give your answer in its simplest form. [Image: A triangle split into 9 equal smaller triangles. 6 of them are shade

In [14]:
train_df["is_mc_answer_correct"] = train_df.Category.str.contains("True", case=False)
train_df.head()

,row_id,QuestionId,QuestionText,MC_Answer,StudentExplanation,Category,Misconception,predict,is_mc_answer_correct
0,0,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),0ne third is equal to tree nineth,True_Correct,NA,True_Correct:NA,True
1,1,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 / 3 because 6 over 9 is 2 thirds and 1 third...,True_Correct,NA,True_Correct:NA,True
2,2,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),"1 3rd is half of 3 6th, so it is simplee to un...",True_Neither,NA,True_Neither:NA,True
3,3,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 goes into everything and 3 goes into nine,True_Neither,NA,True_Neither:NA,True
4,4,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 out of every 3 isn't coloured,True_Correct,NA,True_Correct:NA,True


In [15]:
train_df["is_student_explanation_correct"] = train_df.Category.str.contains("Correct", case=False)
train_df.head()

,row_id,QuestionId,QuestionText,MC_Answer,StudentExplanation,Category,Misconception,predict,is_mc_answer_correct,is_student_explanation_correct
0,0,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),0ne third is equal to tree nineth,True_Correct,NA,True_Correct:NA,True,True
1,1,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 / 3 because 6 over 9 is 2 thirds and 1 third...,True_Correct,NA,True_Correct:NA,True,True
2,2,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),"1 3rd is half of 3 6th, so it is simplee to un...",True_Neither,NA,True_Neither:NA,True,False
3,3,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 goes into everything and 3 goes into nine,True_Neither,NA,True_Neither:NA,True,False
4,4,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 out of every 3 isn't coloured,True_Correct,NA,True_Correct:NA,True,True


In [16]:
train_df["question_str"] = train_df.apply(stringify_input, axis=1)

train_df.question_str.values[:5]

array(["Math Question: What fraction of the shape is not shaded? Give your answer in its simplest form. [Image: A triangle split into 9 equal smaller triangles. 6 of them are shaded.]\nStudent's Answer: \\( \\frac{1}{3} \\)\nStudent's Explanation: 0ne third is equal to tree nineth\nAnswer Correctness: Correct\nExplanation Correctness: Correct\n\nTask: Identify the student's misconception category and specific misconception.",
       "Math Question: What fraction of the shape is not shaded? Give your answer in its simplest form. [Image: A triangle split into 9 equal smaller triangles. 6 of them are shaded.]\nStudent's Answer: \\( \\frac{1}{3} \\)\nStudent's Explanation: 1 / 3 because 6 over 9 is 2 thirds and 1 third is not shaded.\nAnswer Correctness: Correct\nExplanation Correctness: Correct\n\nTask: Identify the student's misconception category and specific misconception.",
       "Math Question: What fraction of the shape is not shaded? Give your answer in its simplest form. [Image: 

In [17]:
max(len(q) for q in train_df.question_str.values)

841